In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold

In [2]:
# train has meter readings, which are obviously lacking in test set
train_df = pd.read_pickle("../data/processed/train_data.pkl")
test_df = pd.read_pickle("../data/processed/test_data.pkl")

In [19]:
train_df.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,...,weekday,area_per_floor,outlier_square_feet,outlier_area_per_floor,air_temperature_6_lag,air_temperature_24_lag,dew_temperature_6_lag,dew_temperature_24_lag,cloud_coverage_6_lag,cloud_coverage_24_lag
0,0,0,0.000000,0,0,8.913550,11.0,NaN,19.406250,3.162209,...,4,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,1142,1,0.000000,13,6,11.549431,NaN,NaN,-7.199219,8.000000,...,4,NaN,False,False,16.768229,18.847656,11.118490,10.296224,3.326116,3.221908
2,1142,0,48.000000,13,6,11.549431,NaN,NaN,-7.199219,8.000000,...,4,NaN,False,False,17.519531,18.801758,11.302083,10.154460,3.072498,3.138575
3,1141,0,21.000000,13,8,9.951611,NaN,NaN,-7.199219,8.000000,...,4,NaN,False,False,18.153646,18.660156,11.402344,9.991862,2.856456,2.971908
4,1140,2,6609.379883,13,0,11.960703,NaN,NaN,-7.199219,8.000000,...,4,NaN,False,False,18.703125,18.518229,11.302083,9.783366,2.856456,2.721908


In [20]:
train_df.shape

(19229841, 26)

In [21]:
train_df.columns

Index(['building_id', 'meter', 'meter_reading', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'relative_humidity', 'air_temp_f', 'feels_like_temp', 'hour', 'weekday',
       'area_per_floor', 'outlier_square_feet', 'outlier_area_per_floor',
       'air_temperature_6_lag', 'air_temperature_24_lag',
       'dew_temperature_6_lag', 'dew_temperature_24_lag',
       'cloud_coverage_6_lag', 'cloud_coverage_24_lag'],
      dtype='object')

In [22]:
test_df.head()

,row_id,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,...,weekday,area_per_floor,outlier_square_feet,outlier_area_per_floor,air_temperature_6_lag,air_temperature_24_lag,dew_temperature_6_lag,dew_temperature_24_lag,cloud_coverage_6_lag,cloud_coverage_24_lag
0,0,0,0,0,0,8.913550,11.0,NaN,15.601562,2.0,...,6,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,0,0,0,7.908387,15.0,NaN,15.601562,2.0,...,6,NaN,True,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,0,0,0,8.589700,28.0,NaN,15.601562,2.0,...,6,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,0,0,0,10.072597,17.0,NaN,15.601562,2.0,...,6,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,0,0,0,11.666565,44.0,NaN,15.601562,2.0,...,6,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
test_df.shape

(41697600, 26)

In [24]:
test_df.columns

Index(['row_id', 'building_id', 'meter', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'relative_humidity', 'air_temp_f', 'feels_like_temp', 'hour', 'weekday',
       'area_per_floor', 'outlier_square_feet', 'outlier_area_per_floor',
       'air_temperature_6_lag', 'air_temperature_24_lag',
       'dew_temperature_6_lag', 'dew_temperature_24_lag',
       'cloud_coverage_6_lag', 'cloud_coverage_24_lag'],
      dtype='object')

In [6]:
groups = train_df.building_id

In [50]:
help(GroupKFold)

Help on class GroupKFold in module sklearn.model_selection._split:

class GroupKFold(_BaseKFold)
 |  GroupKFold(n_splits='warn')
 |  
 |  K-fold iterator variant with non-overlapping groups.
 |  
 |  The same group will not appear in two different folds (the number of
 |  distinct groups has to be at least equal to the number of folds).
 |  
 |  The folds are approximately balanced in the sense that the number of
 |  distinct groups is approximately the same in each fold.
 |  
 |  Parameters
 |  ----------
 |  n_splits : int, default=3
 |      Number of folds. Must be at least 2.
 |  
 |      .. versionchanged:: 0.20
 |          ``n_splits`` default value will change from 3 to 5 in v0.22.
 |  
 |  Examples
 |  --------
 |  >>> import numpy as np
 |  >>> from sklearn.model_selection import GroupKFold
 |  >>> X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
 |  >>> y = np.array([1, 2, 3, 4])
 |  >>> groups = np.array([0, 0, 2, 2])
 |  >>> group_kfold = GroupKFold(n_splits=2)
 |  >>> group_

In [57]:
group_kfold = GroupKFold(n_splits = 5)
group_kfold.get_n_splits()

5

In [58]:
group_kfold.split(train_df, train_df["meter_reading"])

<generator object _BaseKFold.split at 0x0000028464AD6D48>

In [59]:
for train_index, test_index in group_kfold.split(train_df[train_df.meter_reading] == 0, ):
    print("TRAIN:", train_index, "TEST", test_index)

SyntaxError: invalid syntax (<ipython-input-59-f183bdbcf29b>, line 1)

In [13]:
a = "eins"
b = " zwei"
l = False

In [14]:
a + b

'eins zwei'

In [15]:
if l: 
    c = " grouped" 
else:
    c = ""

In [53]:
help(enumerate)

Help on class enumerate in module builtins:

class enumerate(object)
 |  enumerate(iterable, start=0)
 |  
 |  Return an enumerate object.
 |  
 |    iterable
 |      an object supporting iteration
 |  
 |  The enumerate object yields pairs containing a count (from start, which
 |  defaults to zero) and a value yielded by the iterable argument.
 |  
 |  enumerate is useful for obtaining an indexed list:
 |      (0, seq[0]), (1, seq[1]), (2, seq[2]), ...
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [25]:
sum(train_df.meter == 0)

11527365

In [4]:
vec = (train_df.meter == 0).values

In [5]:
test = train_df.iloc[vec,]

In [7]:
test.shape

(11527365, 26)

In [10]:
test2 = train_df[train_df.meter== 0]

In [12]:
test2.shape == test.shape

True

In [44]:
len(vec)

19229841

In [48]:
train_df[train_df.meter == 0]

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,...,weekday,area_per_floor,outlier_square_feet,outlier_area_per_floor,air_temperature_6_lag,air_temperature_24_lag,dew_temperature_6_lag,dew_temperature_24_lag,cloud_coverage_6_lag,cloud_coverage_24_lag
0,0,0,0.000000,0,0,8.913550,11.0,NaN,19.406250,3.162209,...,4,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN
2,1142,0,48.000000,13,6,11.549431,NaN,NaN,-7.199219,8.000000,...,4,NaN,False,False,17.519531,18.801758,11.302083,10.154460,3.072498,3.138575
3,1141,0,21.000000,13,8,9.951611,NaN,NaN,-7.199219,8.000000,...,4,NaN,False,False,18.153646,18.660156,11.402344,9.991862,2.856456,2.971908
6,1140,0,464.627014,13,0,11.960703,NaN,NaN,-7.199219,8.000000,...,4,NaN,False,False,20.286458,18.193034,10.834635,9.416504,3.208615,2.568192
9,1139,0,1350.209961,13,6,12.726799,NaN,NaN,-7.199219,8.000000,...,4,NaN,False,False,23.067708,17.688151,9.882812,8.674642,3.697844,2.257794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19229836,594,0,351.152008,4,0,12.085491,55.0,7.0,NaN,NaN,...,5,10.139581,False,False,17.052083,12.550456,14.348958,11.107422,3.266118,2.814584
19229837,595,0,145.535004,4,0,12.507706,48.0,14.0,NaN,NaN,...,5,9.868648,False,False,16.434896,12.700521,14.299479,11.278320,3.143539,2.936667
19229838,596,0,4.000000,4,0,9.824120,96.0,4.0,NaN,NaN,...,5,8.437825,False,False,15.783854,12.842122,14.115885,11.440755,5.157704,3.579971
19229839,590,0,1.250000,4,0,9.049702,99.0,2.0,NaN,NaN,...,5,8.356555,False,False,15.016927,11.670898,12.782552,10.274089,3.796761,2.290545


In [ ]:
if True:
    print(here)
    for i < 4:
else:
    print(there)
    for i < 3:
        